In [1]:
import sys
import numpy as np

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.autograd import Variable
import Sklearn_PyTorch

import syft as sy

/anaconda3/envs/Pysyft/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Data Preprocessing Functions

In [2]:
from sklearn import preprocessing

### Normalization

In [3]:
def normalize(df): 
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df

### Transforming categorical feature to numerical feature

In [4]:
def encoding(data):
    for col in data.columns:
        if data[col].dtype == type(object):
            le_x = preprocessing.LabelEncoder()
            le_x.fit(data[col])
            data[col] = le_x.transform(data[col])
    return data

### One-Hot Encoding

In [5]:
from sklearn.preprocessing import OneHotEncoder
def label_encoder(df):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(df)
    print(enc.categories_)
    df_array = enc.transform(df).toarray() #Encode the classes to a binary array 
    return df_array

## ACM KDD'99

In [ ]:
data_path = "../../../Dataset/KDD99/kddcup99.csv"

dataset = pd.read_csv(data_path, sep=',', usecols=range(0, 42))

print("Dataset Shape:", dataset.shape)

In [ ]:
#Randomly split data into three parts
data_server = dataset.sample(frac=0.5, random_state=1)
dataset = dataset.drop(data_server.index)
data_alice = dataset.sample(frac=0.5, random_state=1)
data_bob = dataset.drop(data_alice.index)

In [ ]:
#Divide data into x and y
data_server_x = pd.DataFrame(data_server.iloc[:, 0:41])
data_server_y = pd.DataFrame(data_server.iloc[:, 41])
data_alice_x = pd.DataFrame(data_alice.iloc[:, 0:41])
data_alice_y = pd.DataFrame(data_alice.iloc[:, 41])
data_bob_x = pd.DataFrame(data_bob.iloc[:, 0:41])
data_bob_y = pd.DataFrame(data_bob.iloc[:, 41])

In [ ]:
# Relabel data into biclasses
new_class = {'back':'abnormal', 'buffer_overflow':'abnormal', 'ftp_write':'abnormal', 'guess_passwd':'abnormal', 'imap':'abnormal',
            'ipsweep':'abnormal', 'land':'abnormal', 'loadmodule':'abnormal', 'multihop':'abnormal', 'neptune':'abnormal', 'nmap':'abnormal',
            'perl':'abnormal', 'phf':'abnormal', 'pod':'abnormal', 'portsweep':'abnormal', 'rootkit':'abnormal', 'satan':'abnormal',
            'smurf':'abnormal', 'spy':'abnormal', 'teardrop':'abnormal', 'warezclient':'abnormal', 'warezmaster':'abnormal'}
data_server_y = data_server_y.replace(new_class)
data_alice_y = data_alice_y.replace(new_class)
data_bob_y = data_bob_y.replace(new_class)

In [ ]:
# Encode the string datatype to numerial
data_server_x = encoding(data_server_x)
data_server_y = encoding(data_server_y)
data_alice_x = encoding(data_alice_x)
data_alice_y = encoding(data_alice_y)
data_bob_x = encoding(data_bob_x)
data_bob_y = encoding(data_bob_y)

In [ ]:
#Normalize x data
data_server_x = normalize(data_server_x)
data_alice_x = normalize(data_alice_x)
data_bob_x = normalize(data_bob_x)

In [ ]:
#One-Hot encoding labels
data_server_y = label_encoder(data_server_y)
data_alice_y = label_encoder(data_alice_y)
data_bob_y = label_encoder(data_bob_y)

In [ ]:
print(data_server_y)

## IoT Botnet Stream Data

In [6]:
# Load all the data from the CSV file 
BM_DATA_PATH = "../../../Dataset/Botnet_Detection/Philips_B120N10_Baby_Monitor"
DB_DATA_PATH = "../../../Dataset/Botnet_Detection/Danmini_Doorbell"
ET_DATA_PATH = "../../../Dataset/Botnet_Detection/Ecobee_Thermostat"
PT_DATA_PATH = "../../../Dataset/Botnet_Detection/PT_838_Security Camera"
XC_DATA_PATH = "../../../Dataset/Botnet_Detection/XCS7_1002_WHT_Security_Camera"
df_bm_b = pd.read_csv(BM_DATA_PATH+"/benign_traffic.csv")
df_bm_m = pd.read_csv(BM_DATA_PATH+"/Mirai/udp.csv")
df_db_b = pd.read_csv(DB_DATA_PATH+"/benign_traffic.csv")
df_db_m = pd.read_csv(DB_DATA_PATH+"/Mirai/udp.csv")
df_et_b = pd.read_csv(ET_DATA_PATH+"/benign_traffic.csv")
df_et_m = pd.read_csv(ET_DATA_PATH+"/Mirai/udp.csv")
df_pt_b = pd.read_csv(ET_DATA_PATH+"/benign_traffic.csv")
df_pt_m = pd.read_csv(ET_DATA_PATH+"/Mirai/udp.csv")
df_xc_b = pd.read_csv(XC_DATA_PATH+"/benign_traffic.csv")
df_xc_m = pd.read_csv(XC_DATA_PATH+"/Mirai/udp.csv")

In [7]:
#Assign the label to each dataframe
df_bm_b = df_bm_b.assign(label = 'b') 
df_db_b = df_db_b.assign(label = 'b') 
df_et_b = df_et_b.assign(label = 'b')
df_pt_b = df_pt_b.assign(label = 'b')
df_xc_b = df_xc_b.assign(label = 'b')
df_bm_m = df_bm_m.assign(label = 'm')
df_db_m = df_db_m.assign(label = 'm') 
df_et_m = df_et_m.assign(label = 'm') 
df_pt_m = df_pt_m.assign(label = 'm')
df_xc_m = df_xc_m.assign(label = 'm')

In [8]:
print(df_pt_m.shape)
print(df_pt_b.shape)
print(df_xc_m.shape)
print(df_xc_b.shape)

(151481, 116)
(13113, 116)
(151879, 116)
(46585, 116)


In [9]:
#Combine the benign traffic and malicious traffic
df_bm = df_bm_b
df_bm = df_bm.append(df_bm_m, ignore_index = True)
df_db = df_db_b
df_db = df_db.append(df_db_m, ignore_index = True)
df_et = df_et_b
df_et = df_et.append(df_et_m, ignore_index = True)
df_pt = df_pt_b
df_pt = df_pt.append(df_pt_m, ignore_index = True)
df_xc = df_xc_b
df_xc = df_xc.append(df_xc_m, ignore_index = True)

In [10]:
def shuffler(df):
  return df.reindex(np.random.permutation(df.index))

In [11]:
# Shuffle the rows in dataframe
df_bm = shuffler(df_bm)
df_db = shuffler(df_db)
df_et = shuffler(df_et)
df_pt = shuffler(df_pt)
df_xc = shuffler(df_xc)

In [12]:
# Create a dataset on server for initial model (second version)
df_server = pd.DataFrame()
df_server = df_server.append(df_et.sample(frac =.25), ignore_index=True)
df_server = df_server.append(df_pt.sample(frac =.25), ignore_index=True)
df_server = df_server.append(df_xc.sample(frac =.25), ignore_index=True)

In [ ]:
# Create a dataset on server for initial model (first version)
df_server = pd.DataFrame()
df_server = df_server.append(df_bm.sample(frac =.25), ignore_index=True)
df_server = df_server.append(df_db.sample(frac =.25), ignore_index=True)
df_server = df_server.append(df_et.sample(frac =.25), ignore_index=True)

In [13]:
df_server.shape

(131912, 116)

In [14]:
#Divide dataframe into x and y
df_s_x = pd.DataFrame(df_server.iloc[:, 0:115])
df_s_y = pd.DataFrame(df_server.iloc[:, 115])
df_bm_x = pd.DataFrame(df_bm.iloc[:, 0:115])
df_bm_y = pd.DataFrame(df_bm.iloc[:, 115])
df_db_x = pd.DataFrame(df_db.iloc[:, 0:115])
df_db_y = pd.DataFrame(df_db.iloc[:, 115])

In [15]:
#Normalize the x dataframe
df_s_x = normalize(df_s_x)
df_bm_x = normalize(df_bm_x)
df_db_x = normalize(df_db_x)

In [16]:
print(df_bm.shape)

(392274, 116)


In [17]:
print(df_s_x.shape)

(131912, 115)


In [18]:
print(df_db_x.shape)

(287213, 115)


In [19]:
#One-Hot encoding labels and transform into array
s_y = label_encoder(df_s_y)
bm_y = label_encoder(df_bm_y)
db_y = label_encoder(df_db_y)

[array(['b', 'm'], dtype=object)]
[array(['b', 'm'], dtype=object)]
[array(['b', 'm'], dtype=object)]


In [20]:
s_y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

## Start Transfering data to workers 

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [ ]:
class ELM(torch.nn.Module):
    def __init__(self, n_inputs: int, hidden_units = 1000):
        self.random_weights = np.random.normal(size=[n_inputs, hidden_units]) # A random weight is assigned
    
    def learn(self, X: np.ndarray, Y: np.ndarray):
        H = self._hidden_layer(X)
        self.output_weights = np.linalg.pinv(H) @ Y
    
    def _f(self, x: np.ndarray): 
        return 1. / (1. + np.exp(-x)) #activation function: sigmoid
    
    def _hidden_layer(self, inputs: np.ndarray): 
        return self._f(inputs @ self.random_weights)
  
    def _output_layer(self, hidden: np.ndarray): 
        return hidden @ self.output_weights
  
    def __call__(self, inputs: np.ndarray):  #infer
        return self._output_layer(self._hidden_layer(inputs))

In [21]:
class Net(torch.nn.Module):
    def __init__(self, in_dim, h_dim, out_dim):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(in_dim, h_dim)
        self.bn1 = nn.BatchNorm1d(h_dim)
        self.linear2 = torch.nn.Linear(h_dim, out_dim)
        
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [22]:
hook = sy.TorchHook(torch)

### (IoT BotNet)

In [23]:
BM = sy.VirtualWorker(hook, id='BM')
DB = sy.VirtualWorker(hook, id='DB')

In [24]:
from sklearn.model_selection import train_test_split
b_train_x, b_test_x, b_train_y, b_test_y = train_test_split(df_bm_x, bm_y, test_size=0.20)
d_train_x, d_test_x, d_train_y, d_test_y = train_test_split(df_db_x, db_y, test_size=0.20)

In [25]:
tensor_server_x = torch.FloatTensor(df_s_x.values.astype(np.float32))
tensor_server_y = torch.FloatTensor(s_y.astype(np.float32))
t_b_train_x = torch.FloatTensor(b_train_x.values.astype(np.float32))
t_b_test_x = torch.tensor(b_test_x.values.astype(np.float32))
t_b_train_y = torch.tensor(b_train_y.astype(np.float32))
t_b_test_y = torch.tensor(b_test_y.astype(np.float32))
t_d_train_x = torch.tensor(d_train_x.values.astype(np.float32))
t_d_test_x = torch.tensor(d_test_x.values.astype(np.float32))
t_d_train_y = torch.tensor(d_train_y.astype(np.float32))
t_d_test_y = torch.tensor(d_test_y.astype(np.float32))

In [26]:
print(t_d_train_x.shape)
print(t_d_test_x.shape)

torch.Size([229770, 115])
torch.Size([57443, 115])


In [27]:
b_x_train_ptr = t_b_train_x.send(BM)
b_x_test_ptr = t_b_test_x.send(BM)
b_y_train_ptr = t_b_train_y.send(BM)
b_y_test_ptr = t_b_test_y.send(BM)
d_x_train_ptr = t_d_train_x.send(DB)
d_x_test_ptr = t_d_test_x.send(DB)
d_y_train_ptr = t_d_train_y.send(DB)
d_y_test_ptr = t_d_test_y.send(DB)

### (ACM KDD)

In [ ]:
Alice = sy.VirtualWorker(hook, id='Alice')
Bob = sy.VirtualWorker(hook, id='Bob')

In [ ]:
from sklearn.model_selection import train_test_split
a_train_x, a_test_x, a_train_y, a_test_y = train_test_split(data_alice_x, data_alice_y, test_size=0.20)
b_train_x, b_test_x, b_train_y, b_test_y = train_test_split(data_bob_x, data_bob_y, test_size=0.20)

In [ ]:
tensor_server_x = torch.FloatTensor(data_server_x.values.astype(np.float32))
tensor_server_y = torch.FloatTensor(data_server_y.values.astype(np.float32))
t_a_train_x = torch.tensor(a_train_x.values.astype(np.float32))
t_a_test_x = torch.tensor(a_test_x.values.astype(np.float32))
t_a_train_y = torch.tensor(a_train_y.values.astype(np.float32))
t_a_test_y = torch.tensor(a_test_y.values.astype(np.float32))
t_b_train_x = torch.FloatTensor(b_train_x.values.astype(np.float32))
t_b_test_x = torch.tensor(b_test_x.values.astype(np.float32))
t_b_train_y = torch.tensor(b_train_y.values.astype(np.float32))
t_b_test_y = torch.tensor(b_test_y.values.astype(np.float32))

In [ ]:
print(t_b_test_y.shape)

In [ ]:
a_x_train_ptr = t_a_train_x.send(Alice)
a_x_test_ptr = t_a_test_x.send(Alice)
a_y_train_ptr = t_a_train_y.send(Alice)
a_y_test_ptr = t_a_test_y.send(Alice)
b_x_train_ptr = t_b_train_x.send(Bob)
b_x_test_ptr = t_b_test_x.send(Bob)
b_y_train_ptr = t_b_train_y.send(Bob)
b_y_test_ptr = t_b_test_y.send(Bob)

In [ ]:
print(Bob._objects)

In [ ]:
print(Alice._objects)

In [ ]:
print(tensor_server_x)
print(tensor_server_y)

In [ ]:
from Sklearn_PyTorch import TorchRandomForestClassifier

# Initialisation of the model
my_model = TorchRandomForestClassifier(nb_trees=100, nb_samples=3, max_depth=5, bootstrap=True)

# Fitting function
my_model.fit(tensor_server_x, tensor_server_y)


In [ ]:
print(my_model)

## Initialize the parameters

In [28]:
epochs = 600
input_dim = 115
output_dim = 2 #Number of clasees
h_dim = 100
lr_rate = 1e-6

### (Neural Network)

In [29]:
model = Net(input_dim, h_dim, output_dim)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [30]:
def training(epochs, model, data, labels):
    for e in range(int(epochs)):
        y_pred = model(data)

        # Compute and print loss
        loss = criterion(y_pred, labels)
        if e % 100 == 99:
            print(e, loss.item())
    
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [31]:
training(epochs, model, tensor_server_x, tensor_server_y) ## Train the initial model on Server

99 1207.5557861328125
199 455.7712097167969
299 352.61712646484375
399 539.6387329101562
499 464.52520751953125
599 301.3799743652344


### (ELM)

In [ ]:
tensor_server_x.size(0)

In [ ]:
n_hidden = 1000

In [ ]:
model=ELM(tensor_server_x.size(0), n_hidden)

In [ ]:
def training(epochs, model, data, labels):
    

### (Logistic Regression)

In [ ]:
model = LogisticRegression(input_dim, output_dim)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
def training(epochs, model, data, labels):
    for epochs in range(int(epochs)):    
        optimizer.zero_grad() ## Zero out the gradient
        outputs = model(data) ## Call forward
        
        loss = ((outputs - labels)**2).sum() ## softmax
        if epochs % 100 == 99:
            print(loss)
        loss.backward() ## Accumulated gradient updates into x
        optimizer.step()

In [ ]:
tensor_server_y = tensor_server_y.squeeze()
print(tensor_server_y.shape)
training(epochs, model, tensor_server_x, tensor_server_y) ## Train the initial model on Server

## Transfer model to clients

### (IoTBot)

In [32]:
BM_model = model.copy().send(BM)
DB_model = model.copy().send(DB)

BM_opt = torch.optim.SGD(params=BM_model.parameters(),lr=lr_rate)
DB_opt = torch.optim.SGD(params=DB_model.parameters(),lr=lr_rate)

In [33]:
print(t_b_test_y.shape)
print(t_d_test_y.shape)

torch.Size([78455, 2])
torch.Size([57443, 2])


In [34]:
n_bm, y_bm = t_b_test_y.shape

In [35]:
n_db, y_db = t_d_test_y.shape

## Secondary Training on the device with local data

### Neural Network

In [36]:
print(BM._objects)

{43334435263: tensor([[3.1288e-01, 3.1749e-01, 4.8189e-02,  ..., 0.0000e+00, 5.8125e-01,
         4.7004e-01],
        [5.6665e-03, 4.3082e-03, 0.0000e+00,  ..., 5.1075e-04, 5.8129e-01,
         5.2218e-01],
        [2.8535e-01, 2.6712e-01, 9.5405e-02,  ..., 0.0000e+00, 5.8125e-01,
         4.7004e-01],
        ...,
        [2.6337e-01, 1.6125e-01, 1.2721e-01,  ..., 0.0000e+00, 5.8125e-01,
         4.7004e-01],
        [2.2333e-03, 1.0833e-02, 1.3207e-04,  ..., 1.5083e-04, 5.8125e-01,
         4.7202e-01],
        [2.4926e-03, 4.5192e-03, 9.7042e-06,  ..., 4.5933e-04, 5.8125e-01,
         4.6411e-01]]), 23054398979: tensor([[2.2030e-03, 8.5593e-03, 1.2546e-04,  ..., 1.6255e-04, 5.8128e-01,
         5.1509e-01],
        [3.4343e-10, 4.3082e-03, 0.0000e+00,  ..., 5.5477e-04, 5.8125e-01,
         4.6383e-01],
        [4.3966e-01, 1.9566e-01, 1.2690e-01,  ..., 0.0000e+00, 5.8125e-01,
         4.7004e-01],
        ...,
        [3.6867e-01, 2.4843e-01, 1.0766e-01,  ..., 0.0000e+00, 5.8125e-0

In [37]:
print(DB._objects)

{73179098599: tensor([[5.1555e-01, 3.5484e-01, 9.0959e-01,  ..., 0.0000e+00, 1.3573e-01,
         1.4862e-01],
        [4.6970e-01, 3.9699e-01, 9.5279e-01,  ..., 0.0000e+00, 1.3573e-01,
         1.4862e-01],
        [4.6528e-01, 5.2962e-01, 9.9804e-01,  ..., 0.0000e+00, 1.3573e-01,
         1.4862e-01],
        ...,
        [0.0000e+00, 7.8029e-02, 0.0000e+00,  ..., 3.7402e-16, 1.3573e-01,
         1.4862e-01],
        [3.4889e-01, 7.1069e-01, 8.3803e-01,  ..., 0.0000e+00, 1.3573e-01,
         1.4862e-01],
        [4.6934e-01, 3.7863e-01, 9.3574e-01,  ..., 0.0000e+00, 1.3573e-01,
         1.4862e-01]]), 56823256256: tensor([[3.0312e-01, 4.3411e-01, 9.7924e-01,  ..., 0.0000e+00, 1.3573e-01,
         1.4862e-01],
        [1.1648e-02, 8.6243e-02, 5.9663e-17,  ..., 3.7402e-16, 1.3573e-01,
         1.4862e-01],
        [3.2187e-01, 7.8774e-01, 6.9402e-01,  ..., 0.0000e+00, 1.3573e-01,
         1.4862e-01],
        ...,
        [3.6130e-01, 6.4701e-01, 9.2400e-01,  ..., 0.0000e+00, 1.3573e-0

In [38]:
for e in range(100):
    
    #Baby Monitor
    BM_pred = BM_model(b_x_train_ptr)
    
    # Compute and print loss
    BM_loss = criterion(BM_pred, b_y_train_ptr)
    
    # Zero gradients, perform a backward pass, and update the weights.
    BM_opt.zero_grad()
    BM_loss.backward()
    BM_opt.step()
    
    #Door Bell
    DB_pred = BM_model(d_x_train_ptr)
    
    # Compute and print loss
    DB_loss = criterion(DB_pred, d_y_train_ptr)        
    
    # Zero gradients, perform a backward pass, and update the weights.
    DB_opt.zero_grad()
    DB_loss.backward()
    DB_opt.step()
    
    if e % 100 == 99:
        print(e, "BM_loss:", BM_loss.item())
        print(e, "DB_loss:", DB_loss.item())
        total_b = n_bm
        correct = 0
        outputs_b = BM_model(b_x_test_ptr)
        _b, pred_b = torch.max(outputs_b.data, 1)
        vb, labels_b = torch.max(b_y_test_ptr.data, 1)
        correct+= (pred_b == labels_b).sum()
        accuracy_b = 100*correct/total_b
        print("Iteration:", i, "BM Accuracy: ", accuracy_b.get().data)
        total_d = n_db
        correct = 0
        outputs_d = DB_model(d_x_test_ptr)
        _d, pred_d = torch.max(outputs_d.data, 1)
        vd, labels_d = torch.max(d_y_test_ptr.data, 1)
        correct+= (pred_d == labels_d).sum()
        accuracy_d = 100*correct/total_d
        print("Iteration:", i, "DB Accuracy: ", accuracy_d.get().data)

Exception ignored in: <bound method ObjectPointer.__del__ of [PointerTensor | me:42921292518 -> DB:19998143125]>
Traceback (most recent call last):
  File "/anaconda3/envs/Pysyft/lib/python3.6/site-packages/syft/generic/pointers/object_pointer.py", line 220, in __del__
    self.owner.send_msg(ForceObjectDeleteMessage(self.id_at_location), self.location)
  File "/anaconda3/envs/Pysyft/lib/python3.6/site-packages/syft/workers/base.py", line 253, in send_msg
    bin_response = self._send_msg(bin_message, location)
  File "/anaconda3/envs/Pysyft/lib/python3.6/site-packages/syft/workers/virtual.py", line 7, in _send_msg
    return location._recv_msg(message)
  File "/anaconda3/envs/Pysyft/lib/python3.6/site-packages/syft/workers/virtual.py", line 10, in _recv_msg
    return self.recv_msg(message)
  File "/anaconda3/envs/Pysyft/lib/python3.6/site-packages/syft/workers/base.py", line 287, in recv_msg
    response = self._message_router[msg_type](contents)
  File "/anaconda3/envs/Pysyft/lib/py

KeyError: 'Object "69172659437" not found on worker!!!You just tried to interact with an object ID:69172659437 on <VirtualWorker id:BM #objects:12> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

### Logistic Regression

In [ ]:
for i in range(4):

    # Train Bob's Model
    BM_opt.zero_grad()
    BM_pred = BM_model(b_x_train_ptr)
    BM_loss = ((BM_pred - b_y_train_ptr)**2).sum()
    BM_loss.backward()

    BM_opt.step()
    BM_loss = BM_loss.get().data

    # Train Alice's Model
    DB_opt.zero_grad()
    DB_pred = DB_model(d_x_train_ptr)
    DB_loss = ((DB_pred - d_y_train_ptr)**2).sum()
    DB_loss.backward()

    DB_opt.step()
    DB_loss = DB_loss.get().data

    total_b = n_bm
    correct = 0
    outputs_b = BM_model(b_x_test_ptr)
    _b, pred_b = torch.max(outputs_b.data, 1)
    vb, labels_b = torch.max(b_y_test_ptr.data, 1)
    correct+= (pred_b == labels_b).sum()
    accuracy_b = 100*correct/total_b
    print("Iteration:", i, "BM Accuracy: ", accuracy_b.get().data)

    total_d = n_db
    correct = 0
    outputs_d = DB_model(d_x_test_ptr)
    _d, pred_d = torch.max(outputs_d.data, 1)
    vd, labels_d = torch.max(d_y_test_ptr.data, 1)
    correct+= (pred_d == labels_d).sum()
    accuracy_d = 100*correct/total_d
    print("Iteration:", i, "DB Accuracy: ", accuracy_d.get().data)

### (ACM KDD)

In [ ]:
bobs_model = my_model.copy().send(Bob)
alices_model = my_model.copy().send(Alice)

bobs_opt = torch.optim.SGD(params=bobs_model.parameters(),lr=lr_rate)
alices_opt = torch.optim.SGD(params=alices_model.parameters(),lr=lr_rate)

In [ ]:
print(Bob._objects)

In [ ]:
for i in range(2):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(b_x_train_ptr)
    bobs_loss = ((bobs_pred - b_y_train_ptr)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(a_x_train_ptr)
    alices_loss = ((alices_pred - a_y_train_ptr)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data

    total = 24701
    correct = 0
    outputs_a = alices_model(a_x_test_ptr)
    _a, pred_a = torch.max(outputs_a.data, 1)
    va, labels_a = torch.max(a_y_test_ptr.data, 1)
    correct+= (pred_a == labels_a).sum()
    accuracy_a = 100*correct/total
    print("Iteration:", i, "ALice Accuracy: ", accuracy_a.get().data)

    correct = 0
    outputs_b = bobs_model(b_x_test_ptr)
    _b, pred_b = torch.max(outputs_b.data, 1)
    vb, labels_b = torch.max(b_y_test_ptr.data, 1)
    correct+= (pred_b == labels_b).sum()
    accuracy_b = 100*correct/total
    print("Iteration:", i, "Bob Accuracy: ", accuracy_b.get().data)
    

In [ ]:
print(Bob._objects)

In [ ]:
print(Alice._objects)